In [2]:
import numpy as np
import pandas as pd
from neobase import NeoBase

pd.options.display.max_columns = None
#pd.options.display.max_rows = 100

# Exercise 2

**Top 10 arrival airports in the world in 2013 (using the bookings file)**

- To get the total number of passengers for an airport, you can sum the column `pax`, grouping by `arr_port`. Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).
- Print the top 10 arrival airports in the standard output, including the number of passengers.
- **Bonus point**: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at **GeoBases in Github**)
- **Bonus point**: Solve this problem using pandas (instead of any other approach)

### Finished Exercise

We define a function to filter and clean the data:
- Filter by `2013` 
- Drop '`year` column because we don't need it anymore.
- Drop missing values.
- Delete possible blank spaces and make sure all cells are capitalized.
- Finally we add an alert message that shows us if there is any cell with length other than `3`.

In [1]:
def data_preprocessing(df, year=2013):
    '''
    Takes a dataframe and a year, filters it by year, drops the year,
    and drops null values. Cleans the 'arr_port' column.
    Returns a dataframe.
    
    PARAMETERS
    df: Dataframe
    year: Year to filter by. 2013 by default
    '''
    df[df['year'] == year]
    df.drop('year', axis=1, inplace=True)
    df.dropna(inplace=True)
    
    df['arr_port'] = df['arr_port'].str.strip().str.upper()
    
    arr_port_length = (df['arr_port'].map(len) != 3).sum()
    if arr_port_length != 0:
        print('Alert! There are `arr_port` values with length different than 3, check it.')
    
    return df

We read the dataset in chunks. We process each chunk passing it as an argument in our function. Then we aggregate the data and append them to `bookings`. It is not strictly necessary aggregating the data here.

In [3]:
data_iterator = pd.read_csv("../challenge/bookings.csv.bz2", 
                            chunksize=10**6, 
                            sep='^', 
                            usecols=['arr_port', 'pax', 'year'],
                            compression='bz2',
                            #nrows=10**5
                           )

bookings = pd.DataFrame()
for i, data_chunk in enumerate(data_iterator):
    print("Chunk: %d, size of chunk %d"%(i+1, len(data_chunk)))
    
    data_chunk = data_preprocessing(data_chunk)
    chunk_result = data_chunk.pivot_table(values='pax', index='arr_port', 
                                          aggfunc=np.sum)
    #chunk_result = data_chunk.groupby('arr_port').sum()
    
    bookings = bookings.append(chunk_result) 

Chunk: 1, size of chunk 1000000
Chunk: 2, size of chunk 1000000
Chunk: 3, size of chunk 1000000
Chunk: 4, size of chunk 1000000
Chunk: 5, size of chunk 1000000
Chunk: 6, size of chunk 1000000
Chunk: 7, size of chunk 1000000
Chunk: 8, size of chunk 1000000
Chunk: 9, size of chunk 1000000
Chunk: 10, size of chunk 1000000
Chunk: 11, size of chunk 10


Finally we aggregate the data stored in `bookings` and we take the top ten airports by number of passengers.

In [4]:
top_10 = bookings.groupby(by='arr_port').sum().sort_values('pax', ascending=False).head(10)
top_10

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


**Bonus point**: Get the name of the city or airport corresponding to that airport. 

We use NeoBase API to get the data about airports by passing in the IATA code as argument in the `get()` method. We make a list o lists to make it easier if we wanted to add more information to our final dataframe. The `for` loop lets us creating a new column and indicating the field we want to get. We use an `if` statement because sometimes the API give us the information in a list.

In [6]:
elements = [['arr_port_name', 'name'], 
            ['city', 'city_name_list'],
            ['country', 'country_name'],
            ['currency', 'currency'],
           ]

b = NeoBase()
for element in elements:
    if 'list' in element[1]:
        top_10[element[0]] = top_10.index.map(lambda x: b.get(x, element[1])[0])
    else:
        top_10[element[0]] = top_10.index.map(lambda x: b.get(x, element[1]))
top_10

,pax,arr_port_name,city,country,currency
arr_port,,,,,
LHR,88809.0,London Heathrow Airport,London,United Kingdom,GBP
MCO,70930.0,Orlando International Airport,Orlando,United States,USD
LAX,70530.0,Los Angeles International Airport,Los Angeles,United States,USD
LAS,69630.0,McCarran International Airport,Las Vegas,United States,USD
JFK,66270.0,John F. Kennedy International Airport,New York City,United States,USD
CDG,64490.0,Paris Charles de Gaulle Airport,Paris,France,EUR
BKK,59460.0,Suvarnabhumi Airport,Bangkok,Thailand,THB
MIA,58150.0,Miami International Airport,Miami,United States,USD
SFO,58000.0,San Francisco International Airport,San Francisco,United States,USD


In [7]:
top_10.to_csv('exercise_2.csv', sep = ',')

-------------------------------------

### First Approximation

We start by reading a few columns to take a look at the columns and their formats.

In [8]:
bookings = pd.read_csv('../challenge/bookings.csv', nrows=10, sep='^', )
bookings.head()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546


In [9]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   act_date             10 non-null     object
 1   source               10 non-null     object
 2   pos_ctry             10 non-null     object
 3   pos_iata             10 non-null     object
 4   pos_oid              10 non-null     object
 5   rloc                 10 non-null     object
 6   cre_date             10 non-null     object
 7   duration             10 non-null     int64 
 8   distance             10 non-null     int64 
 9   dep_port             10 non-null     object
 10  dep_city             10 non-null     object
 11  dep_ctry             10 non-null     object
 12  arr_port             10 non-null     object
 13  arr_city             10 non-null     object
 14  arr_ctry             10 non-null     object
 15  lst_port             10 non-null     object
 16  lst_city   

`pax` and `year` columns are `int64` data type, so they can't have blank spaces.

Now we know that we will need to use the columns `arr_port`, `pax` and `year`. So we load a bigger sample only with those columns.

In [10]:
bookings = pd.read_csv('../challenge/bookings.csv', nrows=100000, sep='^', 
                       usecols=['arr_port', 'pax', 'year'])
bookings.head()

,arr_port,pax,year
0,LHR,-1,2013
1,CLT,1,2013
2,CLT,1,2013
3,SVO,1,2013
4,SVO,1,2013


We check the columns names and we realized that they must be normalized because there are some blank spaces.

In [11]:
bookings['arr_port'].head().tolist()

['LHR     ', 'CLT     ', 'CLT     ', 'SVO     ', 'SVO     ']

In [12]:
bookings['arr_port'] = bookings['arr_port'].str.strip().str.upper()
bookings['arr_port'].head().tolist()

['LHR', 'CLT', 'CLT', 'SVO', 'SVO']

We aggregate the data just for testing. We realize that we don't need the `year` column in our final solution so we dropped it.

In [13]:
bookings_2013 = bookings[bookings['year'] == 2013]

top_10_2013 = bookings_2013.groupby('arr_port').sum().sort_values(by='pax', ascending=False).head(10)
top_10_2013.drop('year', axis=1)

,pax
arr_port,
LHR,1006
MCO,861
JFK,795
LAX,761
BKK,747
LAS,732
SFO,705
ORD,686
CDG,676


Then we start to build a function to do the cleaning tasks.

In [14]:
def data_preprocessing(df, year):
    '''
    Takes a dataframe and a year, filters it by year, drops the year,
    and drops null values.
    Returns a dataframe.
    
    PARAMETERS
    df: Dataframe
    year: Year to filter by
    '''
    df[df['year'] == year]
    df.drop('year', axis=1, inplace=True)
    df.dropna(inplace=True)
    return df

At this point we start again but loading the data in chunks. We will start by loading the first million rows and we will load the full dataset at the end.

In [15]:
data_iterator = pd.read_csv("../challenge/bookings.csv", chunksize=10**6, 
                            sep='^', usecols=['arr_port', 'pax', 'year'],
                            #nrows=10**6,
                           )

chunk_list = []  
# Each chunk is in dataframe format
for data_chunk in data_iterator: 
    data_chunk = data_preprocessing(data_chunk, 2013)
    chunk_list.append(data_chunk)

bookings = pd.concat(chunk_list)

We can do more cleaning task. The `arr_port` column cells must contain trhee uppercase characteres.

In [17]:
bookings['arr_port'] = bookings['arr_port'].str.strip().str.upper()
bookings['arr_port'].apply(len).value_counts()

3    10000009
Name: arr_port, dtype: int64

In [18]:
top_10 = bookings.pivot_table(values='pax', index='arr_port', 
                              aggfunc=np.sum).sort_values(by='pax', ascending=False).head(10)
top_10

,pax
arr_port,
LHR,88809.0
MCO,70930.0
LAX,70530.0
LAS,69630.0
JFK,66270.0
CDG,64490.0
BKK,59460.0
MIA,58150.0
SFO,58000.0


Finally we use NeoBase to extract data fro each airport.

In [21]:
from neobase import NeoBase

In [22]:
b = NeoBase()

In [23]:
b.get('LHR','city_name_list')

['London']

In [24]:
b.get('LHR','name')

'London Heathrow Airport'

In [25]:
top_10.index[0]

'LHR'

In [26]:
b.get(top_10.index[0]) # My index is 'arr_port'

{'__dup__': set(),
 'iata_code': 'LHR',
 'name': 'London Heathrow Airport',
 'lat': '51.4775',
 'lng': '-0.461389',
 'page_rank': 0.6631884761241168,
 'country_code': 'GB',
 'country_name': 'United Kingdom',
 'continent_name': 'Europe',
 'timezone': 'Europe/London',
 'city_code_list': ['LON'],
 'city_name_list': ['London'],
 'location_type': ['A'],
 'currency': 'GBP'}

In [27]:
b.get(top_10.index[0])['name']

'London Heathrow Airport'

In [28]:
b = NeoBase()
top_10['arr_port_name'] = top_10.index.map(lambda x: b.get(x, 'name'))
top_10['city'] = top_10.index.map(lambda x: b.get(x, 'city_name_list')[0])
top_10['country'] = top_10.index.map(lambda x: b.get(x, 'country_name'))
top_10['currency'] = top_10.index.map(lambda x: b.get(x, 'currency'))
top_10

,pax,arr_port_name,city,country,currency
arr_port,,,,,
LHR,88809.0,London Heathrow Airport,London,United Kingdom,GBP
MCO,70930.0,Orlando International Airport,Orlando,United States,USD
LAX,70530.0,Los Angeles International Airport,Los Angeles,United States,USD
LAS,69630.0,McCarran International Airport,Las Vegas,United States,USD
JFK,66270.0,John F. Kennedy International Airport,New York City,United States,USD
CDG,64490.0,Paris Charles de Gaulle Airport,Paris,France,EUR
BKK,59460.0,Suvarnabhumi Airport,Bangkok,Thailand,THB
MIA,58150.0,Miami International Airport,Miami,United States,USD
SFO,58000.0,San Francisco International Airport,San Francisco,United States,USD


I made some improvements in the code when I prepared the exercise to present it. It is showed at the beginning. 